# X-Ray Segmentation using U-Net

This notebook implements pelvis X-ray segmentation using a U-Net model.

In [5]:
import os
import sys
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Add project root to path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

from src.utils.pengwin_utils import (
    load_image, 
    load_masks, 
    build_augmentation,
    visualize_sample,
    CATEGORIES
)

# Print versions for reproducibility
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.5.1
CUDA available: True
CUDA device: NVIDIA GeForce RTX 4070


## 1. Dataset Implementation
First, let's create our custom dataset class for handling X-ray images and their masks.

In [6]:
class XrayDataset(Dataset):
    def __init__(self, root_dir, split='train', img_size=448):
        self.root = Path(root_dir)
        self.split = split
        self.img_size = img_size
        
        # Setup directories
        self.input_dir = self.root / split / "input" / "images" / "x-ray"
        self.output_dir = self.root / split / "output" / "images" / "x-ray"
        
        # Get file paths
        self.image_paths = sorted(self.input_dir.glob("*.tif"))
        self.mask_paths = sorted(self.output_dir.glob("*.tif"))
        
        assert len(self.image_paths) == len(self.mask_paths)
        
        # Setup augmentation
        self.aug = build_augmentation(train=(split=='train'), img_size=img_size)
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # Load image and mask
        image = load_image(self.image_paths[idx])
        masks, category_ids, _ = load_masks(self.mask_paths[idx])
        
        # Apply augmentation
        augmented = self.aug(image=image, masks=masks)
        
        # Convert to torch tensors
        image = torch.from_numpy(augmented['image']).float()
        masks = torch.from_numpy(np.array(augmented['masks'])).float()
        
        return image, masks

In [7]:
# Initialize dataset
dataset = XrayDataset('path/to/your/data', 'train')
print(f"Dataset size: {len(dataset)}")

# Test loading one sample
image, masks = dataset[0]
print(f"Image shape: {image.shape}")
print(f"Masks shape: {masks.shape}")

# Visualize sample
vis_img = visualize_sample(
    image.numpy(), 
    masks.numpy(), 
    category_ids=list(CATEGORIES.values()),
    fragment_ids=[1]*len(CATEGORIES)
)
plt.figure(figsize=(10, 10))
plt.imshow(vis_img)
plt.axis('off')
plt.show()

Dataset size: 0


IndexError: list index out of range

## 2. U-Net Model Implementation
Now let's implement our U-Net architecture for segmentation.